# Question and answer based on Web Url

## Install Dependencies

In [51]:
%pip install BeautifulSoup4
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install pyperclip
# for web scraping and using langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=cda5c99d063e55584544febd1d90abeef503838cff8c2a74a9ba2fdf9e782a1a
  Stored in directory: /Users/manumishra/Library/Caches/pip/wheels/70/bd/ba/8ae5c080c895c9360fe6e153acda2dee82527374467eae061b
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


## Setup Environment Variables and secrets

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Test setup with basic command

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Sock Explosion.


## Setup Web scraping functions

In [66]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

def get_html(url):
    response = requests.get(url)

    # Ensure the request was successful
    if response.status_code != 200:
        print(f"Failed to get page: {url}")
        return None

    return response.text

def extract_body_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    body_text = soup.body.get_text().strip()
    return body_text

def extract_a_tags(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    a_tags = []
    
    for tag in soup.body.find_all('a'):
        text = tag.get_text().strip()
        href = tag.get('href')
        
        # Only consider tags with non-blank text and href
        if not text or not href:
            continue
        
        full_url = urljoin(base_url, href)
        parsed_url = urlparse(full_url)
        
        # Exclude page anchors
        if parsed_url.path.startswith('#'):
            continue

        # Include relative links and absolute links that belong to the original domain
        if parsed_url.netloc == urlparse(base_url).netloc:
            a_tags.append({'href': full_url, 'text': text})
    
    return a_tags

def extract_information(url):
    html = get_html(url)
    if not html:
        return None
    
    body_text = extract_body_text(html)
    a_tags = extract_a_tags(html, url)
    
    print(f"Extracted body text and {len(a_tags)} a tags from {url}")
    
    additional_body_text = ''
    
    for a_tag in a_tags[:min(10, len(a_tags))]:
        additional_html = get_html(a_tag['href'])
        if additional_html:
            additional_body_text += ' ' + extract_body_text(additional_html)
    
    print(f"Extracted additional body text")
    
    combined_text = body_text + ' ' + additional_body_text
    combined_text = re.sub(r'\s\s+', ' ', combined_text)
    combined_text = re.sub(r'\n+', '\n', combined_text)
    return combined_text

def generate_prompt(a_tags):
    prompt = """Given the following URLs, please reorder them based on the likelihood of containing the following information:
    
    The name of the business.
    The business address.
    A list of services offered by the business along with their prices.
    Please respond with the value in hrefs only, listed in order of likelihood. Do not include additional metadata or explanations. and also remove any duplicates.
    URLs:

    """
    link_str = '\n'.join(str(tag) for tag in a_tags)

    prompt += link_str

    return prompt


#### Test

In [67]:
import pyperclip

testurl = "https://www.solsunless.com/"
testHtml = get_html(testurl)
test_tags = extract_a_tags(testHtml, testurl)
url_prompt = generate_prompt(test_tags)
print(url_prompt)
pyperclip.copy(url_prompt)

from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
text = url_prompt
print(llm(text))

# Join all tags into a single string with newline characters between tags
#tags_str = '\n'.join(str(tag) for tag in test_tags)

# Copy to clipboard
#pyperclip.copy(tags_str)


    

Given the following URLs, please reorder them based on the likelihood of containing the following information:
    
    The name of the business.
    The business address.
    A list of services offered by the business along with their prices.
    Please respond with the value in hrefs only, listed in order of likelihood. Do not include additional metadata or explanations. and also remove any duplicates.
    URLs:

    {'href': 'https://www.solsunless.com/cart', 'text': '0'}
{'href': 'https://www.solsunless.com/contact', 'text': 'Book Now'}
{'href': 'https://www.solsunless.com', 'text': 'HOME'}
{'href': 'https://www.solsunless.com/about', 'text': 'ABOUT'}
{'href': 'https://www.solsunless.com/custom-airbrush-tanning', 'text': 'AIRBRUSH TANNING'}
{'href': 'https://www.solsunless.com/norvell-spray-booth', 'text': 'AUTO BOOTH TANNING'}
{'href': 'https://www.solsunless.com/contact', 'text': 'LOCATIONS'}
{'href': 'https://www.solsunless.com/specials', 'text': 'SPECIALS'}
{'href': 'https://ww

In [30]:
# Call the functions with the URL
url = 'https://www.solsunless.com/'
information = extract_information(url)
print(information)


Extracted body text and 15 a tags from https://www.solsunless.com/
Extracted additional body text
top of page0Book NowHOMEABOUTSERVICESAIRBRUSH TANNINGAUTO BOOTH TANNINGLOCATIONSSPECIALSJOIN OUR TEAMPREPTIPSFAQMY MEMBERSHIPMoreUse tab to navigate through the menu items. WELCOMETO YOUR NEWAnchor 1THE BEST IN SUNLESSBook Now1/1PERFECTBook Now Book Your Flawless TanFLAWLESS AMBASSADORFOLLOW USBecome an Ambassadorarrow&amp;vSubmitSubscribe for exclusive offers©2019 by Sol Sunless Tanning, LLCbottom of page top of page0Book NowHOMEABOUTSERVICESAIRBRUSH TANNINGAUTO BOOTH TANNINGLOCATIONSSPECIALSJOIN OUR TEAMPREPTIPSFAQMY MEMBERSHIPMoreUse tab to navigate through the menu items.My cartCart is emptyContinue Browsingarrow&amp;vSubmitSubscribe for exclusive offers©2019 by Sol Sunless Tanning, LLCbottom of page top of page0Book NowHOMEABOUTSERVICESAIRBRUSH TANNINGAUTO BOOTH TANNINGLOCATIONSSPECIALSJOIN OUR TEAMPREPTIPSFAQMY MEMBERSHIPMoreUse tab to navigate through the menu items.LOCATIONSFrisco2

## Setup Inference chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain

def run_qa_chain(url, prompt_template):
    # Extract information from the URL
    information = extract_information(url)

    print(f"Information length: {len(information)} characters")
    print(f"Information words: {len(information.split())} words")

    # Initialize the OpenAI and QA chains
    llm = OpenAI(model_name = "gpt-3.5-turbo", temperature=0)
    GetDetailedLogs = False
    qa_chain = load_qa_chain(llm, chain_type="refine", verbose=GetDetailedLogs)
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain, verbose=GetDetailedLogs)
    
    # Run the chain and return the result
    result = qa_document_chain.run(input_document=information, question=prompt_template)
    return result

## Experiment

### Prompt 1 [Best]

In [46]:
prompt_template = """
I will give you text from online business website, I want you to answer the following questions about a online business in a JSON format:

Q1. What is the name of the business?
Q2. What is the address of the business?
Q3. What are the services offered by the business?
Q4. What are the service packages and plans provided by the business, What is the price and variations of service package?

Please structure the response in the following JSON format. Also ensure price is always a number:

{
    "NameOfBusiness": "The answer to Q1 should be here",
    "Addresses": [
        "Address1",
        "Address2",
    ],
    "Services offered": [
        "Service1",
        "Service2",
        "Service3"
    ],
    "Packages": [
        {
            "name": "Package1",
            "variation": "variation description",
            "price": "number"
        },
        {
            "name": "Package2",
            "variation": "variation description",
            "price": "number"
        }
    ]
}

"""

### Prompt 2

In [21]:
prompt_template = """answer following questions in provided Json Result format:
Q1. What is the Name of the business?
Q2. What are the Address of the Business?
Q3. What are the different Services offered by this business?
Q4. What are the different service packages with their price information and duration if any?


Result Format:
NameOfBusiness: [answer to Q1]
Addresses: [answer to Q2]
Services offered: [answer to Q3] [name:]
Packages:[answer to Q4] format array of [name:, duration:, price:]
"""

### Prompt 3

In [36]:
prompt_template = """answer following questions in provided:
Question1. What is the Name of the business?
Question2. What are the Address of the Business?
Question3. What are the different services offered by this business?
Question4. What are the different service packages with their price information and duration if any?

"""

In [33]:
prompt_template = """What is the Name of the business?"""

## Run

### Web URL 1 : 'https://www.ritastotalwellness.com/'

In [44]:
url = 'https://www.ritastotalwellness.com/'
print(run_qa_chain(url, prompt_template))

Extracted body text and 23 a tags from https://www.ritastotalwellness.com/
Extracted additional body text


{
    "NameOfBusiness": "Rita's Total Wellness Studio",
    "Addresses": [],
    "Services offered": [
        "Relax, Reset, Revitalize Massage Session",
        "Reiki/ Chakra Balancing Session",
        "Crystal Healing Session",
        "SOUL Coaching/ Intuitive Reading Session",
        "Usui Reiki Healing",
        "Life Coaching",
        "Psychic & Mediumship Development",
        "Meditation",
        "Reiki Infused Healing",
        "Essential Oils & Essential Oil Blends",
        "Reiki Infused Handcrafted Body Care products",
        "Reiki Infused Handcrafted Soy Candles",
        "Handcrafted Healing Crystal Jewelry",
        "Himalayan Salt Hot Stone Massage",
        "Relax, Reset, Revitalize Mobile Massage",
        "Relax, Reset, Revitalize Couples Massage",
        "Relax, Reset, Revitalize Body Scrub",
        "Distance Reiki Healing Session"
    ],
    "Hours

### Web URL 2 : 'https://www.solsunless.com/'

In [47]:
url = 'https://www.solsunless.com/'
print(run_qa_chain(url, prompt_template))

Extracted body text and 15 a tags from https://www.solsunless.com/
Extracted additional body text
Information length: 10762 characters
Information words: 1245 words


{
    "NameOfBusiness": "Sol Sunless Tanning, LLC",
    "Addresses": [
        "8837 Lebanon Road STE 600 Frisco, TX 75034",
        "2704 Cross Timbers Road STE 116 Flower Mound, TX 75028",
        "5555 E. Mockingbird Lane STE 100 Dallas, TX 75206",
        "2910 Cityplace West Blvd STE 120 Dallas, TX 75204",
        "5101McKinney Ranch PKWY STE 110 McKinney, TX 75070"
    ],
    "Services offered": [
        "Airbrush Tanning",
        "Auto Booth Tanning"
    ],
    "Packages": [
        {
            "name": "Flawless Tan",
            "duration": "Varies",
            "price": "Varies"
        },
        {
            "name": "Flawless Ambassador",
            "duration": "Varies",
            "price": "Varies"
        },
        {
            "name": "Single Session",
            "duration": "Varies",
            "

### Web URL 3 : 'https://connectedhealthcenter.com/'

In [48]:
url = 'https://connectedhealthcenter.com/'
print(run_qa_chain(url, prompt_template))

Extracted body text and 11 a tags from https://connectedhealthcenter.com/
Extracted additional body text
Information length: 58211 characters
Information words: 9237 words


{
    "NameOfBusiness": "Connected Health Centers",
    "Addresses": [
        "Phoenix, AZ",
        "Centennial, CO",
        "Colorado Springs, CO",
        "Westminster, CO",
        "Highland Village, TX",
        "Cedar Park, TX",
        "Ventura, CA"
    ],
    "Services offered": [
        "Chiropractic Care",
        "Massage/Rehab",
        "Integrated Medicine",
        "Nutrition",
        "Supplements",
        "Chiropractic Adjustment",
        "Consultation",
        "Diagnostic",
        "Treatment",
        "Education"
    ],
    "Packages": [
        {
            "name": "Chiropractic Care",
            "duration": "Varies",
            "price": "Varies",
            "side effects": "Flu-like symptoms, mild cold, abdominal distress, feeling unexpectedly tired, muscle strain, stiffness, or tensio

### Web URL 4 : 'https://www.perfectionsbodysculpt.com/'

In [68]:
url = 'https://www.perfectionsbodysculpt.com/'
print(run_qa_chain(url, prompt_template))

Extracted body text and 57 a tags from https://www.perfectionsbodysculpt.com/
Extracted additional body text
Information length: 73272 characters
Information words: 11393 words


{
    "NameOfBusiness": "Perfections Body Sculpt",
    "Addresses": [
        "info@perfectionsbodysculpt.com",
        "1017 N Central Expy #200, Suite 105 & 107Plano, TX 75075 | Room 125"
    ],
    "Services offered": [
        "Laser Light Therapy",
        "Ultrasound Cavitation",
        "Laser Lipo Therapy",
        "Brazilian Butt Lift",
        "Red Carpet Laser Facial",
        "EMS Muscle Stimulator",
        "Plasma Fibroblast",
        "Non Invasive Ultrasound Laser Lipo",
        "Vacuum Skin Tighting"
    ],
    "Packages": [
        {
            "name": "Laser Lipo Therapy",
            "duration": "2 sessions a week for 4-6 weeks, 20-60 min per session",
            "price": "Varies depending on the individual's needs",
            "pain": "No pain or bruising involved",
            "recovery